In [2]:
import pandas as pd
import numpy as np
import torch

from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModel
from pythainlp.tokenize import word_tokenize

import re
import unicodedata
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, AutoModel
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import coalesce



pip install --quiet transformers
pip install torch
pip install pandas
pip install pythainlp

In [3]:
# โหลดไฟล์ CSV (กรณีชื่อคอลัมน์ 'label')
df = pd.read_csv('news_dataset_truefakenews.csv')
# ถ้าชื่อคอลัมน์สะกดผิด
df = df.rename(columns={'lable':'label', 'autor':'author'})

In [4]:
# กำหนด mapping dictionary
label_map = {
    'ข่าวจริง': 'true',
    'คลังความรู้': 'true',
    'ข่าวปลอม': 'fake',
    'อาชญากรออนไลน์': 'fake',
    'ข่าวบิดเบือน': 'fake',
    'ข่าวอื่นๆ': None  # หรือจะลบออกเลย
}

In [5]:
# 2) ทำความสะอาด label
ZW = ''.join(['\u200B','\u200C','\u200D','\uFEFF'])

def normalize_thai(s):
    if pd.isna(s):
        return None
    s = str(s).replace('\u00A0',' ').translate({ord(ch):None for ch in ZW})
    s = unicodedata.normalize('NFC', s)
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'([\u0E31\u0E34-\u0E3A\u0E47-\u0E4E])\1+', r'\1', s)
    return s

df['label'] = df['label'].apply(normalize_thai)

# เลือกฉลากมาตรฐาน
CANON = {'ข่าวจริง':'ข่าวจริง', 'ข่าวปลอม':'ข่าวปลอม'}

# map เคสเพี้ยน
ALIASES = {
    'ข่าวจริิง':'ข่าวจริง',
    'ข่่าวจริง':'ข่าวจริง',
    'ข่่าวบิดเบือน':'ข่าวบิดเบือน',
}
df['label'] = df['label'].replace(ALIASES)

# กรองฉลากที่ต้องการและ drop row ที่ไม่มี title/content
df = df[df['label'].isin(CANON.keys())].dropna(subset=['title','content']).reset_index(drop=True)

In [6]:
# สร้างคอลัมน์ใหม่ 'label_binary' เป็น binary label จาก mapping
#df['label_binary'] = df['label'].map(label_map)

# ลบแถวที่เป็น None (ข่าวอื่นๆ)
#df = df.dropna(subset=['label_binary'])

# ถ้าต้องการ save เป็น CSV ใหม่
#df.to_csv('news_dataset_truefakenews.csv', index=False)

# ดูผลลัพธ์
print(df.head())

                  date                                         link  \
0  23/07/2568 09:00:20  https://www.antifakenewscenter.com/?p=74581   
1  23/07/2568 08:00:51  https://www.antifakenewscenter.com/?p=74577   
2  23/07/2568 07:00:51  https://www.antifakenewscenter.com/?p=74573   
3  23/07/2568 06:30:23  https://www.antifakenewscenter.com/?p=74569   
4  23/07/2568 03:30:03  https://www.antifakenewscenter.com/?p=74561   

                                               title  \
0  OR ชวนร่วมเป็นเจ้าของกิจการ ลงทุนหุ้น IPO เริ่...   
1  กระทรวงยุติธรรม เปิดเพจเฟซบุ๊กชื่อ Technology ...   
2  หุ้น OKJ เปิดเทรดวันแรก ราคาเปิดพอร์ตเริ่มต้น ...   
3  กรุงไทย เปิดให้จองสิทธิ์ยืมเงิน 100,000 บาท ลง...   
4            ก.ล.ต. เปิดเว็บไซต์ให้ผู้ลงทุนสะสมคะแนน   

                                             content  \
0  จากที่มีการโฆษณาระบุเปิดให้ลงทุนหุ้น IPO ข้างต...   
1  ระวังเพจปลอมแอบอ้างหน่วยงาน เพจเฟซบุ๊กชื่อ Tec...   
2  เช็กข้อมูลให้ชัวร์ก่อนติดตาม เพจ “OH KAD” เป็น...   
3  การปล่อยส

In [7]:
# 3) ตัดคำ (ไม่เก็บช่องว่าง)
def thai_token_list(text):
    if pd.isna(text):
        return []
    return word_tokenize(text, engine='newmm', keep_whitespace=False)

df['title_tokens'] = df['title'].apply(thai_token_list)
df['content_tokens'] = df['content'].apply(thai_token_list)

In [9]:
# 4) โหลดโมเดลภาษาไทย + จัด device
MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

model = AutoModel.from_pretrained(MODEL_NAME) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = model.to(device).eval()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\Infinix\Documents\GitHub\Project_Thaifakenews\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Infinix\.cache\huggingface\hub\models--airesearch--wangchanberta-base-att-spm-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. I

In [10]:
# Show sample of tokenized data
print("Sample of tokenized titles:")
print(df[['title', 'title_tokens']].head())
print("\nSample of tokenized content:")
print(df[['content', 'content_tokens']].head())

# Print token statistics
print("\nToken statistics:")
print(f"Average tokens in titles: {df['title_tokens'].str.len().mean():.2f}")
print(f"Average tokens in content: {df['content_tokens'].str.len().mean():.2f}")

Sample of tokenized titles:
                                               title  \
0  OR ชวนร่วมเป็นเจ้าของกิจการ ลงทุนหุ้น IPO เริ่...   
1  กระทรวงยุติธรรม เปิดเพจเฟซบุ๊กชื่อ Technology ...   
2  หุ้น OKJ เปิดเทรดวันแรก ราคาเปิดพอร์ตเริ่มต้น ...   
3  กรุงไทย เปิดให้จองสิทธิ์ยืมเงิน 100,000 บาท ลง...   
4            ก.ล.ต. เปิดเว็บไซต์ให้ผู้ลงทุนสะสมคะแนน   

                                        title_tokens  
0  [OR, ชวน, ร่วม, เป็น, เจ้าของกิจการ, ลงทุน, หุ...  
1  [กระทรวงยุติธรรม, เปิด, เพจ, เฟซบุ๊ก, ชื่อ, Te...  
2  [หุ้น, OKJ, เปิด, เทรด, วัน, แรก, ราคา, เปิด, ...  
3  [กรุงไทย, เปิด, ให้, จอง, สิทธิ์, ยืม, เงิน, 1...  
4  [ก.ล.ต., เปิด, เว็บไซต์, ให้, ผู้ลงทุน, สะสม, ...  

Sample of tokenized content:
                                             content  \
0  จากที่มีการโฆษณาระบุเปิดให้ลงทุนหุ้น IPO ข้างต...   
1  ระวังเพจปลอมแอบอ้างหน่วยงาน เพจเฟซบุ๊กชื่อ Tec...   
2  เช็กข้อมูลให้ชัวร์ก่อนติดตาม เพจ “OH KAD” เป็น...   
3  การปล่อยสินเชื่อ บนบัญชี TikTok ชื่อ ktb.thai ..

In [11]:

# โหลด tokenizer และ model ก่อน
model_name = "airesearch/wangchanberta-base-att-spm-uncased"  # ถ้าเป็นภาษาไทยอาจใช้ "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 5) ฟังก์ชัน embedding
@torch.no_grad()
def get_bert_embedding_from_tokens(tokens, max_length=128, use_mean_pool=False):
    text = ' '.join(tokens)
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    ).to(device)
    outputs = model(**inputs)  # last_hidden_state: [1, L, H]
    if use_mean_pool:
        attn = inputs['attention_mask'].unsqueeze(-1)  # [1, L, 1]
        summed = (outputs.last_hidden_state * attn).sum(dim=1)
        denom = attn.sum(dim=1).clamp(min=1)
        emb = (summed / denom).squeeze(0).cpu().numpy()
    else:
        emb = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()  # CLS
    return emb


In [12]:
# 6) สร้าง embedding
use_content = True
title_emb = np.vstack([get_bert_embedding_from_tokens(toks) for toks in df['title_tokens']])
if use_content:
    content_emb = np.vstack([get_bert_embedding_from_tokens(toks) for toks in df['content_tokens']])
    x_np = np.concatenate([title_emb, content_emb], axis=1)
else:
    x_np = title_emb
x_np = normalize(x_np, norm='l2', axis=1)


เสร็จ embedding

In [13]:
# 7) แปลง labels -> ตัวเลข
classes = sorted(df['label'].unique())
label2id = {c:i for i,c in enumerate(classes)}
y_np = df['label'].map(label2id).values
print("Classes:", label2id)

Classes: {'ข่าวจริง': 0, 'ข่าวปลอม': 1}


In [14]:
# 8) สร้าง kNN-graph
N = x_np.shape[0]
k = min(8, max(3, int(np.round(np.log2(max(N, 2))))))
nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(x_np)
dist, idx = nbrs.kneighbors(x_np)

src = np.repeat(np.arange(N), k)
dst = idx[:, 1:].reshape(-1)
w = (1.0 - dist[:, 1:].reshape(-1))

edge_index_np = np.vstack([src, dst])
edge_weight_np = w

# ทำ undirected + coalesce
edge_index_np = np.concatenate([edge_index_np, edge_index_np[::-1, :]], axis=1)
edge_weight_np = np.concatenate([edge_weight_np, edge_weight_np], axis=0)

edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_weight = torch.tensor(edge_weight_np, dtype=torch.float)

# ไม่มี m, n แล้ว
edge_index, edge_weight = coalesce(edge_index, edge_weight, reduce='mean')



In [15]:
N = x_np.shape[0]
k = min(8, max(3, int(np.round(np.log2(max(N, 2))))))  # สูตรง่ายสำหรับ N < 1000

nbrs = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(x_np)
dist, idx = nbrs.kneighbors(x_np)

src = np.repeat(np.arange(N), k)
dst = idx[:, 1:].reshape(-1)
w = (1.0 - dist[:, 1:].reshape(-1))  # cosine similarity ในช่วง [0,1]

edge_index_np = np.vstack([src, dst])
edge_weight_np = w

edge_index_np_rev = edge_index_np[::-1, :]
edge_index_np = np.concatenate([edge_index_np, edge_index_np_rev], axis=1)
edge_weight_np = np.concatenate([edge_weight_np, edge_weight_np], axis=0)

edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_weight = torch.tensor(edge_weight_np, dtype=torch.float)

# เวอร์ชันใหม่ของ PyG ไม่ต้องใช้ m, n
edge_index, edge_weight = coalesce(edge_index, edge_weight, reduce='mean')


In [16]:
# 9) สร้าง Data + split
x = torch.tensor(x_np, dtype=torch.float)
y = torch.tensor(y_np, dtype=torch.long)
data = Data(x=x, edge_index=edge_index, y=y)
data.edge_weight = edge_weight

# split 70/15/15 stratified
idx_all = np.arange(N)
idx_train, idx_test = train_test_split(idx_all, test_size=0.15, stratify=y_np, random_state=42)
idx_train, idx_val = train_test_split(idx_train, test_size=0.1765, stratify=y_np[idx_train], random_state=42)

train_mask = torch.zeros(N, dtype=torch.bool); train_mask[idx_train] = True
val_mask = torch.zeros(N, dtype=torch.bool); val_mask[idx_val] = True
test_mask = torch.zeros(N, dtype=torch.bool); test_mask[idx_test] = True
data.train_mask, data.val_mask, data.test_mask = train_mask, val_mask, test_mask


In [17]:
# 10) สร้าง GCN model
class GCNNet(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hid_dim, normalize=True)
        self.conv2 = GCNConv(hid_dim, out_dim, normalize=True)
        self.dropout = dropout

    def forward(self, data):
        x, ei, ew = data.x, data.edge_index, data.edge_weight
        x = self.conv1(x, ei, edge_weight=ew)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, ei, edge_weight=ew)
        return x

In [18]:
# 11) Train GCN
device_t = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_gnn = GCNNet(in_dim=x.shape[1], hid_dim=128, out_dim=len(classes), dropout=0.5).to(device_t)
data = data.to(device_t)

cls_w = compute_class_weight('balanced', classes=np.arange(len(classes)), y=y_np)
cls_w = torch.tensor(cls_w, dtype=torch.float, device=device_t)

opt = torch.optim.Adam(model_gnn.parameters(), lr=1e-2, weight_decay=5e-4)

def evaluate(logits, mask):
    y_true = data.y[mask].cpu().numpy()
    y_pred = logits[mask].argmax(dim=1).cpu().numpy()
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, f1m

best_val_f1, best_state, patience, counter = -1, None, 30, 0

for epoch in range(1, 301):
    model_gnn.train()
    opt.zero_grad()
    out = model_gnn(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask], weight=cls_w)
    loss.backward(); opt.step()

    model_gnn.eval()
    with torch.no_grad():
        val_acc, val_f1 = evaluate(out, data.val_mask)

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_state = {k: v.detach().cpu().clone() for k,v in model_gnn.state_dict().items()}
        counter = 0
    else:
        counter += 1

    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d} | loss {loss.item():.4f} | val_acc {val_acc:.4f} | val_f1 {val_f1:.4f}")

    if counter >= patience:
        print("Early stopping")
        break

Epoch 020 | loss 0.2657 | val_acc 0.6562 | val_f1 0.5883
Epoch 040 | loss 0.2185 | val_acc 0.6562 | val_f1 0.6102
Early stopping


In [19]:
# load best model
if best_state is not None:
    model_gnn.load_state_dict({k: v.to(device_t) for k,v in best_state.items()})

In [20]:
# evaluate test set
model_gnn.eval()
with torch.no_grad():
    logits = model_gnn(data)
    test_acc, test_f1 = evaluate(logits, data.test_mask)
    print(f"Test Acc: {test_acc:.4f} | Test F1-macro: {test_f1:.4f}")

Test Acc: 0.8750 | Test F1-macro: 0.8333
